# Erklärbare KI

## `Einbindung von externen Wissensbasen`

## Inhaltsverzeichnis



### Beschreibung der Methode

+ Grundlegend: Vorhandensein einer Wissensbasis
+ Erster Schritt: Wissensbasis wird während des Trainings genutzt, um das Modell zu erstellen
+ Zweiter Schritt: Vom KI-Modell getroffene Entscheidungen mit konkreten Einträgen in der Wissensbasis begründen zu können

### Allgemeines

+ Die Ausgabe der KI ist eine Entscheidung und zusätzlich eine Erklärung der Entscheidung
+ Methode ist auf alle Klassifikationsverfahren anwendbar
+ Funktioniert am besten für Klassifikationsprobleme, bei denen Texte als Inputs verwenden werden


### Vorteile
+ Leicht nachvollziehbar 
+ Kombination mehrerer Wissensbasen möglich
+ Erhöht Vertrauen
+ Hilft bei der Frage nach der  Verantwortlichkeit

### Nachteile

+ Abhängig von Qualität der Wissensbasis
+ Aufbau der Wissensbasis zeitaufwendig


### Beispiele

+ Medizinische Anwendung
    + anhand von Symptomen Krankheit vorhersagen und anschließend Vorhersage mit Publikationen unterstützen
+ Home-Automation
+ Bilderkennung


## 1. Imports

In [32]:
import pickle
from rich.console import Console
from rich.table import Table
from tabulate import tabulate
import pandas as pd
from studysearch.run import run as run_studysearch

## 2. Anfrage an das LLM

In [9]:
resp = run_studysearch(input="Worum geht es in dem Studiengang Digitale Technologien?")
print(resp)
pickle.dump(resp, open("./resp.pkl", "wb"))

### 2.2 Import einer gesicherten Antwort

In [22]:
resp = pickle.load(open("resp.pkl", "rb"))

## 3. Ausgabe

In [23]:
def rich_print(resp: dict) -> None:
    """Darstellung des Ergebnisses mittels Rich Text Format.

    Args:
        resp (dict): The response object with query, result and source_documents keys
    """
    table = Table(width=70)
    table.add_column("Query", style="cyan", no_wrap=False)
    table.add_column("Result", style="magenta")
    table.add_column("Source Documents", style="green")
    table.add_row(resp["query"], resp["result"], None)
    for src in resp["source_documents"]:
        table.add_row(None, None, f"{src.metadata['source']}\n")
    console = Console()
    console.print(table)

In [24]:
rich_print(resp)

┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ Query                ┃ Result               ┃ Source Documents     ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ Worum geht es in dem │ The study program    │                      │
│ Studiengang Digitale │ "Digitale            │                      │
│ Technologien?        │ Technologien         │                      │
│                      │ (praxisintegriert)"  │                      │
│                      │ focuses on the       │                      │
│                      │ practical            │                      │
│                      │ application of       │                      │
│                      │ digital technologies │                      │
│                      │ in various fields,   │                      │
│                      │ such as software     │                      │
│                      │ development, data    │                      │
│                      │ analytics,           │                      │
│                      │ cybersecurity, and   │                      │
│                      │ more. The course     │                      │
│                      │ aims to provide      │                      │
│                      │ students with a      │                      │
│                      │ comprehensive        │                      │
│                      │ understanding of the │                      │
│                      │ theoretical          │                      │
│                      │ foundations of       │                      │
│                      │ digital technologies │                      │
│                      │ and their practical  │                      │
│                      │ applications in      │                      │
│                      │ real-world           │                      │
│                      │ scenarios.           │                      │
│                      │                      │ data/Studiengangspr… │
│                      │                      │ und Modulhandbuch    │
│                      │                      │ Bachelor Digitale    │
│                      │                      │ Technologien         │
│                      │                      │ praxisintegriert     │
│                      │                      │ Version 18.pdf       │
│                      │                      │                      │
│                      │                      │ data/Studiengangspr… │
│                      │                      │ und Modulhandbuch    │
│                      │                      │ Bachelor Digitale    │
│                      │                      │ Technologien         │
│                      │                      │ praxisintegriert     │
│                      │                      │ Version 18.pdf       │
│                      │                      │                      │
└──────────────────────┴──────────────────────┴──────────────────────┘

In [50]:
resp["source_documents"] = [src.metadata['source'] for src in resp["source_documents"]]
resp_df = pd.DataFrame(resp).rename(columns={"query": "Query", "result": "Result", "source_documents": "Source Documents"})
for i in range(1, len(resp["source_documents"])):
    resp_df.loc[i, "Query"] = ""
    resp_df.loc[i, "Result"] = ""
resp_df.drop_duplicates(subset=["Source Documents"], inplace=True, keep="first")
# resp_df = resp_df.set_index(["source_documents"])
tabulate(resp_df, tablefmt="html", headers="keys", showindex=False)

Query,Result,Source Documents
Worum geht es in dem Studiengang Digitale Technologien?,"The study program ""Digitale Technologien (praxisintegriert)"" focuses on the practical application of digital technologies in various fields, such as software development, data analytics, cybersecurity, and more. The course aims to provide students with a comprehensive understanding of the theoretical foundations of digital technologies and their practical applications in real-world scenarios.",data/Studiengangsprüfungsordnung und Modulhandbuch Bachelor Digitale Technologien praxisintegriert Version 18.pdf
